# Image Classifier Training Pipeline
## Data Augmentation, Class Balancing & 2-Layer CNN

This notebook demonstrates the complete training pipeline for a fruit image classifier with:
- **Data Augmentation**: Rotation, zoom, brightness adjustments, etc.
- **Class Balancing**: Handles imbalanced fruit categories
- **Simplified Architecture**: Max 2 convolutional layers with Gaussian noise regularization

## Step 1: Import Required Libraries

In [1]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input, GaussianNoise
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json

print("✓ All libraries imported successfully!")

ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (c:\Users\skido\anaconda3\Lib\site-packages\keras\preprocessing\image\__init__.py)

## Step 2: Create Data Augmentation Generators

The "Confusion" Generator creates new variations of your training photos on the fly to help the model learn better.

In [ ]:
# --- 1. DATA AUGMENTATION (The "Confusion" Generator) ---
# This creates new variations of your photos on the fly.
train_datagen = ImageDataGenerator(
    rescale=1./255,                    # Normalize pixel values
    rotation_range=40,                 # Tilt photo up to 40 degrees
    width_shift_range=0.2,             # Shift left/right
    height_shift_range=0.2,            # Shift up/down
    shear_range=0.2,                   # Distort shape (shear)
    zoom_range=0.2,                    # Zoom in/out
    horizontal_flip=True,              # Mirror image
    brightness_range=[0.8, 1.2],       # Simulate different lighting
    channel_shift_range=20.0,          # Slight color changes (simulates background tint)
    fill_mode='nearest'
)

# Test data should NOT be augmented, only scaled.
test_datagen = ImageDataGenerator(rescale=1./255)

print("✓ Data augmentation generators created!")
print("\nAugmentation parameters:")
print("  - Rotation: ±40°")
print("  - Shift: ±20% (width & height)")
print("  - Zoom: ±20%")
print("  - Brightness: 0.8 - 1.2x")
print("  - Horizontal flip: Yes")

## Step 3: Load Data Generators

Load training and test data from directory structure with augmentation applied.

In [ ]:
# Load Data
train_generator = train_datagen.flow_from_directory(
    'TeamX/data/train',               # Point to your training folder
    target_size=(150, 150),
    batch_size=16,                    # Small batch size for small data
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    'TeamX/data/test',
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical'
)

print(f"✓ Data generators loaded!")
print(f"\nTraining data:")
print(f"  - Total batches: {len(train_generator)}")
print(f"  - Classes: {list(train_generator.class_indices.keys())}")
print(f"  - Class indices: {train_generator.class_indices}")

print(f"\nTest data:")
print(f"  - Total batches: {len(test_generator)}")
print(f"  - Classes: {list(test_generator.class_indices.keys())}")

## Step 4: Compute Class Weights

Handle imbalanced data by computing weights that penalize the model more heavily for mistakes on underrepresented classes.

In [ ]:
# --- 2. BALANCING (Handling unequal amounts of data) ---
# If 'Apple' has 70 photos and 'Mixed' has 20, this calculates weights
# so the model is penalized more for getting 'Mixed' wrong.
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
# Convert to dictionary format required by Keras
class_weight_dict = dict(enumerate(class_weights))

print("✓ Class weights computed!")
print("\nClass Weight Distribution:")
for class_name, class_idx in train_generator.class_indices.items():
    weight = class_weight_dict[class_idx]
    print(f"  {class_name}: {weight:.4f}")

## Step 5: Build Model Architecture

Create a simplified CNN with **exactly 2 convolutional layers**, Gaussian noise for regularization, and dropout for preventing overfitting.

In [ ]:
# --- 3. ARCHITECTURE (Add Noise, Remove Complexity) ---
model = Sequential()

# Input Layer + Gaussian Noise (Artificial Static)
model.add(Input(shape=(150, 150, 3)))
# This adds random noise to training data to prevent memorization
model.add(GaussianNoise(0.1))

# Convolution Block 1
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Convolution Block 2 (Max 2 convolutional layers)
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten and Dense Layers
model.add(Flatten())

# Dropout: Randomly sets 50% of inputs to 0.
# This forces the model to not rely on specific paths.
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(Dense(4, activation='softmax'))  # 4 classes: Apple, Orange, Banana, Mixed

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

print("✓ Model created and compiled!")
print("\nModel Architecture:")
model.summary()

## Step 6: Setup Training Callbacks

Configure callbacks for:
- Early stopping (prevent overfitting)
- Learning rate reduction (adaptive learning)
- Model checkpointing (save best model)

In [ ]:
# Create experiment directory
experiment_dir = Path('experiments/notebook_demo')
experiment_dir.mkdir(parents=True, exist_ok=True)

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        filepath=str(experiment_dir / 'model_best.h5'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

print("✓ Callbacks configured!")
print(f"✓ Experiment directory: {experiment_dir}")

## Step 7: Train the Model

**⏱️ NOTE:** Training will take several minutes depending on your hardware. The model will train with:
- **Data augmentation** applied to training data on-the-fly
- **Class weights** to balance imbalanced fruit categories
- **Early stopping** to prevent overfitting
- **Learning rate reduction** for adaptive optimization

In [ ]:
# --- 4. COMPILING AND TRAINING ---
print("Starting training with Class Weights:", class_weight_dict)
print("\nTraining configuration:")
print(f"  - Epochs: 50")
print(f"  - Batch size: 16")
print(f"  - Learning rate: 0.001")
print(f"  - Class weights: {class_weight_dict}")
print(f"  - Data augmentation: Enabled")
print("\n" + "="*70 + "\n")

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=50,
    validation_data=test_generator,
    validation_steps=len(test_generator),
    class_weight=class_weight_dict,  # Apply the balancing here
    callbacks=callbacks,
    verbose=1
)

print("\n✓ Training completed!")

## Step 8: Visualize Training History

Plot the training and validation accuracy/loss over epochs.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Plot accuracy
axes[0].plot(history.history['accuracy'], label='Training Accuracy', marker='o')
axes[0].plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
axes[0].set_title('Model Accuracy', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot loss
axes[1].plot(history.history['loss'], label='Training Loss', marker='o')
axes[1].plot(history.history['val_loss'], label='Validation Loss', marker='s')
axes[1].set_title('Model Loss', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(experiment_dir / 'training_history.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Training history plots saved!")

## Step 9: Evaluate Model on Test Set

Generate predictions and compute metrics.

In [ ]:
# Get predictions
y_pred_proba = model.predict(test_generator)
y_pred = y_pred_proba.argmax(axis=1)
y_test = test_generator.classes

# Calculate accuracy
final_accuracy = accuracy_score(y_test, y_pred)

print("✓ Evaluation completed!")
print(f"\n🎯 Final Test Accuracy: {final_accuracy:.4f} ({final_accuracy*100:.2f}%)")

# Get class names
class_names = list(test_generator.class_indices.keys())
class_names_sorted = sorted(class_names, key=lambda x: test_generator.class_indices[x])

# Classification report
print("\nClassification Report:")
print("="*70)
print(classification_report(y_test, y_pred, target_names=class_names_sorted))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

## Step 10: Visualize Confusion Matrix

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=class_names_sorted,
    yticklabels=class_names_sorted,
    cbar_kws={'label': 'Count'}
)
plt.title('Confusion Matrix - Test Set', fontsize=12, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig(experiment_dir / 'confusion_matrix.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Confusion matrix visualization saved!")

## Step 11: Save Training History to JSON

In [ ]:
# Save training history
history_dict = {
    'accuracy': history.history['accuracy'],
    'val_accuracy': history.history['val_accuracy'],
    'loss': history.history['loss'],
    'val_loss': history.history['val_loss']
}

with open(experiment_dir / 'history.json', 'w') as f:
    json.dump(history_dict, f, indent=4)

# Save metrics
metrics_dict = {
    'final_accuracy': float(final_accuracy),
    'final_accuracy_percent': float(final_accuracy * 100),
    'test_samples': int(len(y_test)),
    'class_distribution': {name: int(sum(y_test == test_generator.class_indices[name])) 
                          for name in class_names_sorted}
}

with open(experiment_dir / 'metrics.json', 'w') as f:
    json.dump(metrics_dict, f, indent=4)

print("✓ Training history saved to history.json")
print("✓ Metrics saved to metrics.json")

## Summary

✅ **Training Pipeline Completed!**

### Key Features Implemented:
- ✓ **Data Augmentation** (rotation, zoom, brightness, shifts)
- ✓ **Class Balancing** (handles imbalanced fruit categories)
- ✓ **Simplified Architecture** (exactly 2 convolutional layers)
- ✓ **Gaussian Noise** (prevents overfitting/memorization)
- ✓ **Dropout Regularization** (50% drop rate)
- ✓ **Early Stopping** (prevents overfitting)
- ✓ **Learning Rate Scheduling** (adaptive optimization)

### Output Files:
- `model_best.h5` - Best trained model (saved via checkpoint)
- `history.json` - Training/validation metrics per epoch
- `metrics.json` - Final accuracy and class distribution
- `training_history.png` - Accuracy & loss plots
- `confusion_matrix.png` - Confusion matrix visualization

### Model Architecture:
```
Input (150×150×3) → Gaussian Noise (0.1)
  ↓
Conv2D(32, 3×3) + ReLU → MaxPool(2×2)
  ↓
Conv2D(64, 3×3) + ReLU → MaxPool(2×2)
  ↓
Flatten → Dropout(0.5) → Dense(512, ReLU) → Dense(4, Softmax)
```

**Next Steps:** You can now use this trained model for predictions on new fruit images!

In [ ]:
# Update paths since we're running from TeamX/src
import os
os.chdir('../../')  # Go to root directory for data access